# Homework 3

## FINM 35700 - Spring 2023

### UChicago Financial Mathematics

* Aman Krishna
* aman.krishna@uchicago.edu

This homework relies on:

- the US government bonds symbology file `govt_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the corporate  bonds symbology file `corp_symbology`,
- the market data file `market_prices_eod` and
- the corporate bonds call schedule file `call_schedules`.


# Problem 1: More details on callable bonds: "workout-date" and "yield-to-worst" calculations

In [130]:
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt
import os
import matplotlib.pyplot as plt
import random

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Use static calculation/valuation date of 2023-04-14, matching data available in the market prices EOD file
calc_date = ql.Date(14, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date

mydir = os.getcwd()

In [131]:
# Calculate initial term and current time-to-maturity for each bond issue
def get_symbology(df, underlying=False):
    for index, row in df.iterrows():
        start_date = ql.Date(row['start_date'].day, row['start_date'].month, row['start_date'].year)
        maturity_date = ql.Date(row['maturity'].day, row['maturity'].month, row['maturity'].year)
        today_date = ql.Date(14,4,2023)
        calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
        #set dcc as Actual/365.25
        dcc = ql.Actual36525()
        initial_term = dcc.yearFraction(start_date, maturity_date)
        current_time_to_maturity = dcc.yearFraction(today_date, maturity_date)
        df.at[index, 'term'] = initial_term
        df.at[index, 'TTM'] = current_time_to_maturity

    if underlying:
        return df[['ticker','figi','security','name','und_bench_isin','coupon','days_settle','start_date','acc_first', 'maturity', 'term', 'TTM']]
    else:
        #round term to integer
        df['term'] = round(df['term'],2)
        df1 = df[['ticker','figi','security','name','coupon','days_settle','start_date','acc_first', 'maturity', 'term', 'TTM']]
        return df1

In [132]:
def get_ql_date(date) -> ql.Date:
    """
    convert dt.date to ql.Date
    """
    if isinstance(date, dt.date):
        return ql.Date(date.day, date.month, date.year)
    elif isinstance(date, str):
        date = dt.datetime.strptime(date, "%Y-%m-%d").date()
        return ql.Date(date.day, date.month, date.year)
    else:
        raise ValueError(f"to_qldate, {type(date)}, {date}")

In [133]:
def create_schedule_from_symbology(details: dict):
    '''Create a QuantLib cashflow schedule from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    # Create maturity from details['maturity']
    maturity = ql.Date(details['maturity'].day, details['maturity'].month, details['maturity'].year)
    
    # Create acc_first from details['acc_first']
    acc_first = ql.Date(details['acc_first'].day, details['acc_first'].month, details['acc_first'].year)
    
    # Create calendar for Corp and Govt asset classes
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    
    # define period from details['cpn_freq'] ... can be hard-coded to 2 = semi-annual frequency
    period = ql.Period(ql.Semiannual)
    
    # business_day_convention
    business_day_convention = ql.Unadjusted
    
    # termination_date_convention
    termination_date_convention = ql.Unadjusted
    
    # date_generation
    date_generation=ql.DateGeneration.Backward
    
    # Create schedule using ql.MakeSchedule interface (with keyword arguments)
    schedule = ql.MakeSchedule(effectiveDate=acc_first,  # this may not be the same as the bond's start date
                            terminationDate=maturity,
                            tenor=period,
                            calendar=calendar,
                            convention=business_day_convention,
                            terminalDateConvention=termination_date_convention,
                            rule=date_generation,
                            endOfMonth=True,
                            firstDate=ql.Date(),
                            nextToLastDate=ql.Date())
    return schedule

In [134]:
def create_bond_from_symbology(details: dict):
    '''Create a US fixed rate bond object from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    
     # Create day_count from details['dcc']
     # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
     # For US Corporates use ql.Thirty360(ql.Thirty360.USA)
     
    if details['ticker'] == 'T':
        day_count = ql.ActualActual(ql.ActualActual.ISMA)
    else:
        day_count = ql.Thirty360(ql.Thirty360.USA)
    
    # Create day_count from details['start_date']    
    issue_date = ql.Date(details['start_date'].day, details['start_date'].month, details['start_date'].year)
    
    # Create days_settle from details['days_settle']
    days_settle = int(float(details['days_settle']))

    # Create days_settle from details['coupon']
    coupon = float(details['coupon'])/100.

    # Create cashflow schedule
    schedule = create_schedule_from_symbology(details)
    
    face_value = 100
    redemption = 100
    
    payment_convention = ql.Unadjusted
        
    # Create fixed rate bond object
    fixed_rate_bond = ql.FixedRateBond(
        days_settle,
        face_value,
        schedule,
        [coupon],
        day_count,
        payment_convention,
        redemption,
        issue_date)        

    return fixed_rate_bond


## a. Load and explore the call schedules dataframe

Load the `call_schedules` Excel file into a dataframe. It contains call schedules for fixed-rate, callable corporate bonds.

For each bond in the dataframe, compute the total number of outstanding calls.

In [135]:
call_schedules = pd.read_excel(os.path.join(mydir, 'data', 'call_schedules.xlsx'))

In [136]:
call_schedules

,isin,figi,call_date,call_price
0,US254687CZ75,BBG00QNKJ092,2024-06-15,100
1,US254687DD54,BBG00QNKPCL5,2025-07-15,100
2,US254687DK97,BBG00QNKGJP6,2026-08-15,100
3,US254687EX00,BBG00QNKKP22,2045-04-15,100
4,US254687FB70,BBG00QNKKP68,2046-05-15,100
...,...,...,...,...
110,US92346MHV63,BBG012F6PR25,2031-03-15,100
111,US92346MJS17,BBG0175X1628,2023-05-15,100
112,US92346MJZ59,BBG017QYK9Z0,2023-06-15,100
113,US92346MKG59,BBG0191HMXJ3,2023-08-15,100


In [137]:
#group by isin and count number of call_date
call_schedules['call_count'] = call_schedules.groupby('isin')['call_date'].transform('count')
call_schedules

,isin,figi,call_date,call_price,call_count
0,US254687CZ75,BBG00QNKJ092,2024-06-15,100,1
1,US254687DD54,BBG00QNKPCL5,2025-07-15,100,1
2,US254687DK97,BBG00QNKGJP6,2026-08-15,100,1
3,US254687EX00,BBG00QNKKP22,2045-04-15,100,1
4,US254687FB70,BBG00QNKKP68,2046-05-15,100,1
...,...,...,...,...,...
110,US92346MHV63,BBG012F6PR25,2031-03-15,100,16
111,US92346MJS17,BBG0175X1628,2023-05-15,100,1
112,US92346MJZ59,BBG017QYK9Z0,2023-06-15,100,1
113,US92346MKG59,BBG0191HMXJ3,2023-08-15,100,1


## b. Load symbology dataframe and extend it with the fixed-rate callable corporate bond details from 1a.

Load the `corp_symbology` Excel file into a dataframe and keep only the fixed-rate callable bonds from 1a.

Extend the dataframe with the column computed in 1a:

| num_call_dates |
|----------|

In [138]:
corp_symbology = pd.read_excel(os.path.join(mydir, 'data', 'corp_symbology.xlsx'))
corp_symbology

,ticker,class,figi,isin,trace,und_bench_isin,security,name,type,coupon,...,cpn_freq,days_settle,start_date,cpn_first,acc_first,maturity,mty_typ,rank,country,currency
0,DIS,Corp,BBG00QNKJ092,US254687CZ75,DIS4921182,US91282CGU99,DIS 3.7 09/15/24,WALT DISNEY COMPANY/THE,GLOBAL,3.700,...,2,2,2019-11-22,2020-03-15,2019-09-15,2024-09-15,CALLABLE,Sr Unsecured,US,USD
1,DIS,Corp,BBG00QNKPCL5,US254687DD54,DIS4907444,US91282CGU99,DIS 3.7 10/15/25,WALT DISNEY COMPANY/THE,GLOBAL,3.700,...,2,2,2019-11-22,2020-04-15,2019-10-15,2025-10-15,CALLABLE,Sr Unsecured,US,USD
2,DIS,Corp,BBG00QNKGJP6,US254687DK97,DIS4907439,US91282CGR60,DIS 3 3/8 11/15/26,WALT DISNEY COMPANY/THE,GLOBAL,3.375,...,2,2,2019-11-22,2020-05-15,2019-11-15,2026-11-15,CALLABLE,Sr Unsecured,US,USD
3,DIS,Corp,BBG00QNKP8R8,US254687DV52,DIS4908605,US91282CGM73,DIS 6.55 03/15/33,WALT DISNEY COMPANY/THE,GLOBAL,6.550,...,2,2,2019-11-22,2020-03-15,2019-09-15,2033-03-15,AT MATURITY,Sr Unsecured,US,USD
4,DIS,Corp,BBG00QNKR4J4,US254687DZ66,DIS4908062,US91282CGM73,DIS 6.2 12/15/34,WALT DISNEY COMPANY/THE,GLOBAL,6.200,...,2,2,2019-11-22,2019-12-15,2019-06-15,2034-12-15,AT MATURITY,Sr Unsecured,US,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,VZ,Corp,BBG017BR3G75,US92346MJT99,VZ5414106,US91282CGT27,VZ 4.15 05/15/29,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.150,...,2,2,2022-05-19,2022-11-15,2022-05-19,2029-05-15,CALLABLE,Sr Unsecured,US,USD
261,VZ,Corp,BBG017QYK9Z0,US92346MJZ59,VZ5420578,US912810TL26,VZ 4.65 06/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.650,...,2,2,2022-06-03,2022-12-15,2022-06-03,2052-06-15,CALLABLE,Sr Unsecured,US,USD
262,VZ,Corp,BBG0191HMXJ3,US92346MKG59,VZ5458130,US912810TL26,VZ 4.6 08/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.600,...,2,2,2022-08-11,2023-02-15,2022-08-11,2052-08-15,CALLABLE,Sr Unsecured,US,USD
263,VZ,Corp,BBG019LXC9P3,US92346MKQ32,VZ5477070,US91282CGT27,VZ 4 1/2 09/15/29,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.500,...,2,2,2022-09-22,2023-03-15,2022-09-22,2029-09-15,CALLABLE,Sr Unsecured,US,USD


In [139]:
corp_symbology_call = corp_symbology[corp_symbology['cpn_type'] == 'FIXED']
corp_symbology_call = corp_symbology_call.reset_index(drop=True)

#merge corp_symbology with call_schedules and keep only bonds in call_schedules
corp_symbology_call = corp_symbology_call.merge(call_schedules, how='inner', on='isin')

#rename figi_x to figi and drop figi_y
corp_symbology_call = corp_symbology_call.rename(columns={'figi_x':'figi'})
corp_symbology_call = corp_symbology_call.drop(columns=['figi_y'])

In [140]:
corp_symbology_call

,ticker,class,figi,isin,trace,und_bench_isin,security,name,type,coupon,...,cpn_first,acc_first,maturity,mty_typ,rank,country,currency,call_date,call_price,call_count
0,DIS,Corp,BBG00QNKJ092,US254687CZ75,DIS4921182,US91282CGU99,DIS 3.7 09/15/24,WALT DISNEY COMPANY/THE,GLOBAL,3.700,...,2020-03-15,2019-09-15,2024-09-15,CALLABLE,Sr Unsecured,US,USD,2024-06-15,100,1
1,DIS,Corp,BBG00QNKPCL5,US254687DD54,DIS4907444,US91282CGU99,DIS 3.7 10/15/25,WALT DISNEY COMPANY/THE,GLOBAL,3.700,...,2020-04-15,2019-10-15,2025-10-15,CALLABLE,Sr Unsecured,US,USD,2025-07-15,100,1
2,DIS,Corp,BBG00QNKGJP6,US254687DK97,DIS4907439,US91282CGR60,DIS 3 3/8 11/15/26,WALT DISNEY COMPANY/THE,GLOBAL,3.375,...,2020-05-15,2019-11-15,2026-11-15,CALLABLE,Sr Unsecured,US,USD,2026-08-15,100,1
3,DIS,Corp,BBG00QNKKP22,US254687EX00,DIS4920157,US912810TQ13,DIS 4.95 10/15/45,WALT DISNEY COMPANY/THE,GLOBAL,4.950,...,2020-04-15,2019-10-15,2045-10-15,CALLABLE,Sr Unsecured,US,USD,2045-04-15,100,1
4,DIS,Corp,BBG00QNKKP68,US254687FB70,DIS4907442,US912810TQ13,DIS 4 3/4 11/15/46,WALT DISNEY COMPANY/THE,GLOBAL,4.750,...,2020-05-15,2019-11-15,2046-11-15,CALLABLE,Sr Unsecured,US,USD,2046-05-15,100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,VZ,Corp,BBG012F6PR25,US92346MHV63,VZ5256196,US912810TL26,VZ 3.05 09/15/51,VERIZON COMMUNICATIONS,DOMESTIC MTN,3.050,...,2022-03-15,2021-09-16,2051-09-15,CALLABLE,Sr Unsecured,US,USD,2031-03-15,100,16
111,VZ,Corp,BBG0175X1628,US92346MJS17,VZ5410330,US912810TL26,VZ 4.55 05/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.550,...,2022-11-15,2022-05-12,2052-05-15,CALLABLE,Sr Unsecured,US,USD,2023-05-15,100,1
112,VZ,Corp,BBG017QYK9Z0,US92346MJZ59,VZ5420578,US912810TL26,VZ 4.65 06/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.650,...,2022-12-15,2022-06-03,2052-06-15,CALLABLE,Sr Unsecured,US,USD,2023-06-15,100,1
113,VZ,Corp,BBG0191HMXJ3,US92346MKG59,VZ5458130,US912810TL26,VZ 4.6 08/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.600,...,2023-02-15,2022-08-11,2052-08-15,CALLABLE,Sr Unsecured,US,USD,2023-08-15,100,1


> Please note as per the interprepation for this question, I did an innder join between the dataframes. This would mean that there is one bond with 16 different call dates and the rest are uniques bonds with one call date each.

## c. Add a function to compute "workout date" and "yield-to-worst" for a fixed-rate callable bond

Quick recap: given the current market price, the issuer is expected to call a callable bond on the call date corresponding to lowest bond yield "to call date" (which is the best possible scenario from the point of view of the issuer exercising the call option). The lowest yield for a call date is called the "yield-to-worst" and the corresponding call date (on which the issuer is expected to call the bond) is called the "workout date". Keep in mind that the "workout date" could be at the bond maturity date, in which case "yield-to-worst" = "yield-to-maturity".

To do: For a callable bond, with known symbology (reference data) and call schedules dataframes, create a function that takes the clean market price as an input and returns "workout date" and "yield-to-worst".

Internally, for each call date, create a separate "call scenario bond" object using the function create_bond_from_symbology() from Homework 2, and then compute the corresponding "call scenario yield" for the "call scenario bond" (using the bond clean market price).


In [141]:
def calc_yield_to_worst(
            details: dict,
            pc_schedule: pd.DataFrame,
            bond_clean_price: float,
            calc_date: ql.Date):
    '''Computes yield-to-worst and workout date for fixed rate callable bonds.
    '''
    
    # Create bond object from symbology details
    bond = create_bond_from_symbology(details)
    
    # Create call schedule from pc_schedule
    call_schedule = create_schedule_from_pc_schedule(pc_schedule)

    
    # update code!!!
    workout_date = ql.Date()    # compute workout date !!!
    yield_to_worst = 0.05       # compute yield to worst !!!    
    
    return workout_date, yield_to_worst

## d. Compute "workout dates" and "yields-to-worst" for all fixed-rate callable bonds

Load the `market_prices_eod` Excel file into a dataframe, which contains marktet quotes as of 2023-04-14.

For each fixed-rate callable bond in the symbology dataframe from 1b (using calc_date = '2023-04-14' and clean market prices):
- 1. Compute the yield to maturity 

- 2. Use the function from 1c to compute "workout date" and "yield-to-worst".

Extend the symbology dataframe with the following columns:


| clean_price |  yield_to_maturity | workout_date | yield_to_worst |
|----------|-------------|-------|-------------|



In [142]:
market_data = pd.read_excel(os.path.join(mydir, 'data', 'market_prices_eod.xlsx'))

In [143]:
market_data.head()

,date,class,ticker,isin,figi,bid,ask,mid_clean,mid_dirty,bid_yield,ask_yield
0,2023-04-14,Corp,DIS,US254687CZ75,BBG00QNKJ092,98.678,98.848,98.763,99.102,4.679,4.551
1,2023-04-14,Corp,DIS,US254687DD54,BBG00QNKPCL5,98.454,98.821,98.637,98.668,4.361,4.203
2,2023-04-14,Corp,DIS,US254687DK97,BBG00QNKGJP6,97.090,97.369,97.230,98.664,4.260,4.174
3,2023-04-14,Corp,DIS,US254687DV52,BBG00QNKP8R8,115.163,115.984,115.574,116.174,4.624,4.529
4,2023-04-14,Corp,DIS,US254687DZ66,BBG00QNKR4J4,112.939,113.586,113.262,115.380,4.742,4.674


In [144]:
# Use static calculation/valuation date of 2023-04-14, matching data available in the market prices EOD file
calc_date = ql.Date(14, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date

In [145]:
corp_symbology_call_md = corp_symbology_call.merge(market_data, how='left', on='figi')

corp_symbology_call_md = corp_symbology_call_md.rename(columns={'class_x':'class', 'ticker_x':'ticker', 'isin_x':'isin','mid_clean':'clean_price'})
corp_symbology_call_md = corp_symbology_call_md.drop(columns=['class_y', 'ticker_y', 'isin_y','bid', 'ask', 'mid_dirty','bid_yield', 'ask_yield'])

#### Function to compute yield to maturity

In [146]:
#define a function to calculate the yield to maturity for each bond using QuantLib
def calc_yield_to_maturity(details: dict, bond_clean_price: float, calc_date: ql.Date):
    '''Computes yield-to-maturity for fixed rate bonds.
    '''
    # Create bond object
    bond = create_bond_from_symbology(details)
    
    # Create bond engine
    bond_engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(ql.FlatForward(calc_date, 0.05, ql.ActualActual(ql.ActualActual.ISMA))))
    
    # Set bond engine
    bond.setPricingEngine(bond_engine)
    
    # Compute yield-to-maturity
    yield_to_maturity = bond.bondYield(bond_clean_price, ql.ActualActual(ql.ActualActual.ISMA), ql.Compounded, ql.Semiannual)
    
    return yield_to_maturity

In [147]:
corp_symbology_call_md['yield_to_maturity'] = corp_symbology_call_md.apply(lambda x: calc_yield_to_maturity(x, x['clean_price'], calc_date), axis=1) * 100

In [148]:
corp_symbology_call_md

,ticker,class,figi,isin,trace,und_bench_isin,security,name,type,coupon,...,mty_typ,rank,country,currency,call_date,call_price,call_count,date,clean_price,yield_to_maturity
0,DIS,Corp,BBG00QNKJ092,US254687CZ75,DIS4921182,US91282CGU99,DIS 3.7 09/15/24,WALT DISNEY COMPANY/THE,GLOBAL,3.700,...,CALLABLE,Sr Unsecured,US,USD,2024-06-15,100,1,2023-04-14,98.763,4.617400
1,DIS,Corp,BBG00QNKPCL5,US254687DD54,DIS4907444,US91282CGU99,DIS 3.7 10/15/25,WALT DISNEY COMPANY/THE,GLOBAL,3.700,...,CALLABLE,Sr Unsecured,US,USD,2025-07-15,100,1,2023-04-14,98.637,4.282175
2,DIS,Corp,BBG00QNKGJP6,US254687DK97,DIS4907439,US91282CGR60,DIS 3 3/8 11/15/26,WALT DISNEY COMPANY/THE,GLOBAL,3.375,...,CALLABLE,Sr Unsecured,US,USD,2026-08-15,100,1,2023-04-14,97.230,4.217668
3,DIS,Corp,BBG00QNKKP22,US254687EX00,DIS4920157,US912810TQ13,DIS 4.95 10/15/45,WALT DISNEY COMPANY/THE,GLOBAL,4.950,...,CALLABLE,Sr Unsecured,US,USD,2045-04-15,100,1,2023-04-14,98.660,5.050273
4,DIS,Corp,BBG00QNKKP68,US254687FB70,DIS4907442,US912810TQ13,DIS 4 3/4 11/15/46,WALT DISNEY COMPANY/THE,GLOBAL,4.750,...,CALLABLE,Sr Unsecured,US,USD,2046-05-15,100,1,2023-04-14,96.513,5.003430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,VZ,Corp,BBG012F6PR25,US92346MHV63,VZ5256196,US912810TL26,VZ 3.05 09/15/51,VERIZON COMMUNICATIONS,DOMESTIC MTN,3.050,...,CALLABLE,Sr Unsecured,US,USD,2031-03-15,100,16,2023-04-14,62.167,5.775920
111,VZ,Corp,BBG0175X1628,US92346MJS17,VZ5410330,US912810TL26,VZ 4.55 05/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.550,...,CALLABLE,Sr Unsecured,US,USD,2023-05-15,100,1,2023-04-14,86.643,5.472957
112,VZ,Corp,BBG017QYK9Z0,US92346MJZ59,VZ5420578,US912810TL26,VZ 4.65 06/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.650,...,CALLABLE,Sr Unsecured,US,USD,2023-06-15,100,1,2023-04-14,87.994,5.478433
113,VZ,Corp,BBG0191HMXJ3,US92346MKG59,VZ5458130,US912810TL26,VZ 4.6 08/15/52,VERIZON COMMUNICATIONS,DOMESTIC MTN,4.600,...,CALLABLE,Sr Unsecured,US,USD,2023-08-15,100,1,2023-04-14,87.750,5.438779


# Problem 2: Perpetual bonds
## a. Price a fixed rate perpetual bond
We are interested in a fixed rate perpetual bond (infinite maturity) on a face notional of $100 and semi-annual coupon c.

Assuming that the bond has a (continuously componded) yield of y, what is the fair value price of the bond?

For simplicity, you can assume T+0 settlement and zero accrued.

## b. Perpetual bonds priced "at par"
For which yield y does the bond trade "at par", i.e. fair value price = $100?

## c. Duration and DV01 for a fixed rate perpetual bond
Compute Duration and DV01 of the perpetual bond.

## d. Convexity of a fixed rate perpetual bond
Compute the convexity of the perpetual bond.

# Problem 3: US SOFR swap curve calibration as of 2023-04-28
### Follow Section "11. SOFR OIS swap rates and SOFR discount curve calibration + validation" in the QuantLib Intro notebook !!!

## a. Load and explore US SOFR swaps symbology and market data

Load the `sofr_swap_symbology` Excel file into a dataframe. Print all swap tenors available.

Load the `sofr_swaps_market_data_eod` Excel file into a dataframe. Print all dates available.

Plot the historial time series of SOFR rates for the available [1Y, 2Y, 3Y, 5Y, 7Y, 10Y, 20Y, 30Y] tenors.

## b. Calibrate the US SOFR yield curve (via bootstrapping)
The function below shows how to calibrate a smooth yield/discount factor curve from SOFR swaps. 

Prepare a joint symbology & market dataframe quotes as of 2023-04-28. 

Calibrate the SOFR discount factor curve as of 2023-04-28.

Follow section 11b.

In [149]:
def calibrate_sofr_curve_from_frame(
        calc_date: ql.Date,
        sofr_details: pd.DataFrame,
        rate_quote_column: str):
    '''Create a calibrated yield curve from a SOFR details dataframe which includes rate quotes.
    '''
    ql.Settings.instance().evaluationDate = calc_date

    # Sort dataframe by maturity
    sorted_details_frame = sofr_details.sort_values(by='tenor')    
    
    # settle_days
    settle_days = 2
    
    # For US SOFR OIS Swaps 
    day_count = ql.Actual360()

    # For US SOFR Swaps     
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    
    sofr_helpers = []
    
    for index, row in sorted_details_frame.iterrows():
        sofr_quote = row[rate_quote_column]
        tenor_in_years = row['tenor']
        sofr_tenor = ql.Period(tenor_in_years, ql.Years)
        
        # create sofr_rate_helper
        sofr_helper = ql.OISRateHelper(settle_days, sofr_tenor, ql.QuoteHandle(ql.SimpleQuote(sofr_quote/100)), ql.Sofr())
                        
        sofr_helpers.append(sofr_helper)
        
    sofr_yield_curve = ql.PiecewiseLinearZero(settle_days, calendar, sofr_helpers, day_count)
    sofr_yield_curve.enableExtrapolation()
    
    return sofr_yield_curve

## c. Display the calibrated SOFR discount curve dataframe

Follow section 11d to display the calibration details dataframe.

## d. Plot the calibrated US SOFR Zero Interest Rates and Discount Factor curves

Plot the SOFR zero rates and discount factor curves by maturity. Follow section 11c.

# Problem 4: CDS Hazard Rate calibration and valuation
## Follow Section "12. CDS Hazard Rate calibration + Pricing" in the QuantLib Intro notebook !!!

## a. Load and explore the CDS market data (IBM credit issuer)

Load the `cds_market_data_eod` Excel file into a dataframe. 

Plot the historical time series of CDS Par Spreads for the available tenors.


## b. Calibrate the IBM hazard rate curve as of 2023-04-28

Follow section 12a. Use the calibrated SOFR discount curve from 2b.

## c. Plot the calibrated Hazard Rates and Survival Probability curves
Follow section 12b. Use the calibrated SOFR discount curve from 2b.

## d. Compute the fair/par spread and PV of a CDS 

Follow section 12c. Construct a CDS object with 100 bps coupon and 2027-06-20 maturity. Compute the fair/par spread and PV.


